In [6]:
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


In [7]:
#!pip install fastparquet

In [8]:
#!pip install pyarrow

In [9]:
target_action = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']

In [10]:
#df = pd.read_csv('data/ga_hits.csv')

df = pd.read_csv('data/ga_hits_exp.csv')


In [11]:
df['target_action'] = df['event_action'].apply(lambda x: 1 if x in target_action else 0)
df[df['target_action'] == 1].head(5)


,Unnamed: 0,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value,target_action
84,4078743,7916978149655436745.1623093770.1623093770,2021-06-07,NaN,22,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/bf24b977?rental_page=rental_car,sub_button_click,sub_callback_submit_click,KclpemfoHstknWHFiLit,NaN,1
341,12165927,7287821919621059379.1632736219.1632736219,2021-09-27,NaN,13,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/2fc745ed?utm_source_initial=...,sub_button_click,sub_car_claim_click,EsLbNNEnCkXWoaesnKlS,NaN,1
469,9607732,6883705466031047854.1630882063.1630882063,2021-09-06,NaN,54,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/145d53f1?utm_source_initial=...,sub_button_click,sub_car_claim_click,nzDXtAvtTrzfwscIPQkp,NaN,1
491,9677106,3370328772300720201.1631799369.1631799369,2021-09-16,NaN,17,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/e647f545?utm_source_initial=...,sub_button_click,sub_car_claim_click,EsLbNNEnCkXWoaesnKlS,NaN,1
713,9267245,2834246583796328852.1628137876.1628137876,2021-08-05,NaN,5,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,sub_button_click,sub_call_number_click,KclpemfoHstknWHFiLit,NaN,1


In [12]:
#df_reduced = df.sample(frac=0.05)

In [13]:
df2 = pd.read_csv('data/ga_sessions.csv')

C:\Users\user\AppData\Local\Temp\ipykernel_9588\2530114789.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('data/ga_sessions.csv')


In [14]:
df2.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.162225,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.162225,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [15]:
#df2.to_parquet('data/ga_hits_exp.parquet')

In [16]:
#df_rdcd = df.sample(frac=50000 / 786324)


In [17]:
#df_reduced.to_csv('data/ga_hits_exp.csv')

In [18]:
df3 = pd.merge(df, df2, on='session_id')
df3.head()

,Unnamed: 0,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,...,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,15153284,6496257600138814417.1639228370.1639228370,2021-12-11,58381.0,18,event,NaN,sberauto.com/cars?city=1&rental_page=rental_on...,sap_search_form,sap_search_form_cost_to,...,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
1,3355953,6496257600138814417.1639228370.1639228370,2021-12-11,63792.0,31,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,card_web,view_new_card,...,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,375x667,Safari,Russia,Moscow
2,5686332,2099932520499870787.1622114044.1622114044,2021-05-27,NaN,7,event,HbolMJUevblAbkHClEQa,sberauto.com/cars?city=1&rental_car=rental_only,search_form,search_form_mark_select,...,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Xiaomi,NaN,360x780,Chrome,Russia,Saint Petersburg
3,8273786,4626314043010719631.1628315541.1628315541,2021-08-07,NaN,28,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/faq,sub_page_view,sub_faq,...,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,414x896,Safari,Russia,Moscow
4,11622233,6496742338732522867.1633975666.1633975666,2021-10-11,NaN,31,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/all/mercedes-benz/e-klasse/d...,card_web,view_card,...,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,428x926,Safari,Russia,Moscow


In [19]:
def cols_to_drop_list_obvious(df3):
    print('% of empties')
    cols_to_drop = []
    for col in df3.columns:
        print(col, ' - ', round(len(df3[col][df3[col].isna() == True]) / len(df3[col]), 2))
        if len(df3[col][df3[col].isna() == True]) / len(df3[col]) > 0.9:
            cols_to_drop.append(col)
    return cols_to_drop

In [20]:
def show_empties_percentage(df3):
    print('% of empties')
    #cols_to_drop = []
    for col in df3.columns:
        print(col, ' - ', round(len(df3[col][df3[col].isna() == True]) / len(df3[col]), 3))
    b = 0
    return b

In [21]:
cols_to_drop = cols_to_drop_list_obvious(df3)

% of empties
Unnamed: 0  -  0.0
session_id  -  0.0
hit_date  -  0.0
hit_time  -  0.59
hit_number  -  0.0
hit_type  -  0.0
hit_referer  -  0.39
hit_page_path  -  0.0
event_category  -  0.0
event_action  -  0.0
event_label  -  0.24
event_value  -  1.0
target_action  -  0.0
client_id  -  0.0
visit_date  -  0.0
visit_time  -  0.0
visit_number  -  0.0
utm_source  -  0.0
utm_medium  -  0.0
utm_campaign  -  0.14
utm_adcontent  -  0.18
utm_keyword  -  0.59
device_category  -  0.0
device_os  -  0.59
device_brand  -  0.25
device_model  -  0.99
device_screen_resolution  -  0.0
device_browser  -  0.0
geo_country  -  0.0
geo_city  -  0.0


In [22]:
df3['new_date'] = pd.to_datetime(df['hit_date'])
cols_to_drop.append('hit_date')


In [23]:
df3['day_of_week'] = df3.new_date.dt.dayofweek

In [24]:
cols_to_drop.append('new_date')

In [25]:
cols_to_drop.append('session_id')

In [26]:
cols_to_drop.append('hit_time')

In [27]:
df3.hit_type.unique()
cols_to_drop.append('hit_type')

In [28]:
cols_to_encode = []
len(df3.hit_referer.unique())
cols_to_encode.append('hit_referer')

In [29]:
df3.loc[df3.hit_referer.isna(), 'hit_referer'] = 'other'

In [30]:
df3['hit_page_path_greenday'] = df3['hit_page_path'].apply(lambda x: 1 if 'green' in x else 0)
df3['hit_page_path_subscription'] = df3['hit_page_path'].apply(lambda x: 1 if 'podpiska' in x else 0)
df3['hit_page_path_general'] = df3['hit_page_path'].apply(lambda x: 0 if 'podpiska' in x else (0 if 'green' in x else 1))



In [31]:
cols_to_drop.append('hit_page_path')

In [32]:
df3.event_category.unique()

array(['sap_search_form', 'card_web', 'search_form', 'sub_page_view',
       'listing_ads', 'cars', 'sub_button_click', 'quiz', 'auth',
       'main_banners', 'chat', 'active_invitation', 'sap_search__form',
       'jivosite', 'greenday_sub_button_click', 'offers', 'ad_banner',
       'navigation', 'sub_submit', 'header', 'footer', 'add_ad',
       'profile_menu', 'scrolling_main_page', 'pos_landing',
       'greenday_listing_ads', 'my_idea', 'credit_landing_pos', 'phone',
       'main_services', 'greenday_sub_submit'], dtype=object)

In [33]:
cols_to_encode.append('event_category')


In [34]:
cols_to_drop.append('event_action')


In [35]:
df3.loc[df3.event_label.isna(), 'event_label'] = 'other'
#df3[df3.event_label.isna() == True] = 'other'

In [36]:
cols_to_encode.append('event_label')

In [37]:
cols_to_drop.append('event_value')

In [38]:
df3['client_number_of_visits'] = 1

In [39]:
#df3 = df3.drop('client_number_of_visits', axis=1)

In [40]:
counter = 0
for cl_id in df3[df3.client_id.duplicated() == True].client_id:
    
    df3.loc[df3.client_id == cl_id, 'client_number_of_visits'] = len(df3[df3.client_id == cl_id])

In [41]:
cols_to_drop.append('client_id')

In [42]:
cols_to_drop.append('visit_date')

In [43]:
cols_to_encode.append('utm_source')

In [44]:
cols_to_encode.append('utm_medium')

In [45]:
show_empties_percentage(df3)

% of empties
Unnamed: 0  -  0.0
session_id  -  0.0
hit_date  -  0.0
hit_time  -  0.587
hit_number  -  0.0
hit_type  -  0.0
hit_referer  -  0.0
hit_page_path  -  0.0
event_category  -  0.0
event_action  -  0.0
event_label  -  0.0
event_value  -  1.0
target_action  -  0.0
client_id  -  0.0
visit_date  -  0.0
visit_time  -  0.0
visit_number  -  0.0
utm_source  -  0.0
utm_medium  -  0.0
utm_campaign  -  0.141
utm_adcontent  -  0.181
utm_keyword  -  0.589
device_category  -  0.0
device_os  -  0.586
device_brand  -  0.253
device_model  -  0.993
device_screen_resolution  -  0.0
device_browser  -  0.0
geo_country  -  0.0
geo_city  -  0.0
new_date  -  0.0
day_of_week  -  0.0
hit_page_path_greenday  -  0.0
hit_page_path_subscription  -  0.0
hit_page_path_general  -  0.0
client_number_of_visits  -  0.0


0

In [46]:
df3.utm_campaign.unique()

array(['LEoPHuyFvzoNfnzGgfcd', 'LTuZkdKfxRGVceoWkVyg',
       'wpmatvDuiMutnvmNVPJo', 'UvuMsOSDBWQGOIbDbXfV', nan,
       'FTjNLDyTrXaWYgZymFkV', 'eimRuUrNhZLAYcwRrNXu',
       'hkvDVxqLOzGjGaoNiNzN', 'SgIUDYUKnyWHVowUOqid',
       'gVwpGdJDpteNmFIatPTG', 'LUlvACDKkkOkiSuiwaBs',
       'gecBYcKZCPMcVYdSSzKP', 'YCKgTzTDywjcWyQudGch',
       'vRLTQQKAsFQMxbcHGPhu', 'PXQWdUxeUoXfoKzTBGpY',
       'jvsVVQVKRfBIqBDBuCkq', 'kVOrIKZFrEYGvixPclal',
       'foFTSdUvNqqkPzZvgiqt', 'KgicpPxiEQfzPlPwQZJq',
       'XHNUiSKKGTEpSAlaVMFQ', 'sbJRYgVfvcnqKJNDDYIr',
       'CdkIkBrvEVomSgvHzYvj', 'zxoiLxhuSIFrCeTLQVWZ',
       'ngkgBNjlzLYBofkljaBo', 'LwJZxKuWmvOhPsaCeRjG',
       'bxOTvPtyGSdUrbwoXCPO', 'QEejXfOCtOMFLZqIPApp',
       'GWZGdHKPgmZPNPwkJshU', 'jpjTolpOezlrOPAunDZi',
       'iYBYglGljMDRQyqHRiPH', 'vXsFkagGabkcWKlgLzSg',
       'emlBaVkgLVRqbEVOSSXt', 'VBmazutCflYumtDHrQYe',
       'JwYIveaHVpeeRZloQCfF', 'zPJpddwzkFqLMSYgtDqy',
       'mrrsABvDmholqItQOtcL', 'okTXSMadDkjvntEHzIjp',
     

In [47]:
len(df3[df3.utm_campaign.isna() == True])

6974

In [48]:
df3.loc[df3.utm_campaign.isna(), 'utm_campaign'] = 'other'

In [49]:
utm_c_frec_dict = {}
for pos in df3.utm_campaign.unique():
    if len(df3[(df3.utm_campaign == pos) & (df3.target_action == 1)]) == 0:
        utm_c_frec_dict[str(pos)] = 0
    else:
        utm_c_frec_dict[str(pos)] = round(len(df3[(df3.utm_campaign == pos) & (df3.target_action == 1)]) / len(df3[df3.utm_campaign == pos]), 5)

In [50]:
df3['camp_succ_rate'] = df3.utm_campaign.apply(lambda x: utm_c_frec_dict[str(x)])

In [51]:
cols_to_drop.append('utm_campaign')

In [52]:
cols_to_drop.append('utm_adcontent')

In [53]:
#len(df3[df3.utm_keyword.isna() == True])
cols_to_drop.append('utm_keyword')

In [54]:
df3.device_category.unique()

array(['mobile', 'desktop', 'tablet'], dtype=object)

In [55]:
cols_to_encode.append('device_category')

In [56]:
def show_how_many_empties_in_feature(df, feature_name):
    
    return len(df[df[feature_name].isna() == True])

In [57]:
show_how_many_empties_in_feature(df3,'device_os')

28980

In [58]:
df3.loc[df3.device_os.isna(), 'device_os'] = 'unknown'

In [59]:
cols_to_encode.append('device_os')

In [60]:
df3.device_brand

0          Apple
1          Apple
2         Xiaomi
3          Apple
4          Apple
          ...   
49408    Samsung
49409    Samsung
49410     Xiaomi
49411      Apple
49412      Apple
Name: device_brand, Length: 49413, dtype: object

In [61]:
show_how_many_empties_in_feature(df3, 'device_brand')

12504

In [62]:
df3.loc[df3.device_brand.isna(), 'device_brand'] = 'unknown'


In [63]:
cols_to_encode.append('device_brand')

In [64]:
cols_to_drop.append('device_screen_resolution')

In [65]:
show_how_many_empties_in_feature(df3, 'device_model')

49067

In [66]:
cols_to_drop.append('device_model')

In [67]:
tst_l = []
for i in range(len(df3.device_category)):
    tst_l.append((df3.device_category[i], eval(df3.device_screen_resolution[i].replace('x','*'))))
len(tst_l)

49413

In [68]:
df3['resolution'] = df3.device_screen_resolution.apply(lambda x:eval(x.replace('x','*')))

In [69]:
bounds = []

for device in df3.device_category.unique():
    q = df3[df3.device_category == device].resolution.quantile()
    iqr = q * 2
    lower_bound = q * 1 - 1.5 * iqr
    upper_bound = q * 3 + 1.5 * iqr
    bounds.append((device, lower_bound, upper_bound))
bounds

[('mobile', -655872.0, 1967616.0),
 ('desktop', -3528000.0, 10584000.0),
 ('tablet', -1572864.0, 4718592.0)]

In [70]:
resolution = []
for i in range(len(tst_l)):
    if tst_l[i][0] == bounds[0][0]:
        resolution.append(bounds[0][0]+'_high' if tst_l[i][1] >= bounds[0][2] * 0.7 else (bounds[0][0]+'_medium' if bounds[0][2] * 0.7 > tst_l[i][1] >= bounds[0][2] * 0.1 else bounds[0][0]+'_low'))
    elif tst_l[i][0] == bounds[1][0]:
        resolution.append(bounds[1][0]+'_high' if tst_l[i][1] >= bounds[1][2] * 0.7 else (bounds[1][0]+'_medium' if bounds[1][2] * 0.7 > tst_l[i][1] >= bounds[1][2] * 0.1 else bounds[1][0]+'_low'))
    elif tst_l[i][0] == bounds[2][0]:
        resolution.append(bounds[2][0]+'_high' if tst_l[i][1] >= bounds[2][2] * 0.7 else (bounds[2][0]+'_medium' if bounds[2][2] * 0.7 > tst_l[i][1] >= bounds[2][2] * 0.1 else bounds[2][0]+'_low'))
        

In [71]:
df3.resolution = resolution

In [72]:
shares = []
for i in df3.resolution.unique():
    print(i)
    shares.append(len(df3[df3.resolution == str(i)]))
shares

mobile_medium
desktop_medium
tablet_low
desktop_low
mobile_low
tablet_medium
mobile_high
desktop_high
tablet_high


[35828, 10074, 69, 2415, 626, 346, 40, 14, 1]

In [73]:
cols_to_encode.append('resolution')

In [74]:
cols_to_encode.append('device_browser')

In [75]:
for item in df3.geo_country.unique():
    if len(df3[df3.geo_country == item]) / len(df3) >= 0.002:
        continue
    else:
        df3.loc[df3.geo_country == item, 'geo_country'] = 'some_unimportant_country'
           

In [76]:
for item in df3.geo_country.unique():
    print(item, ' - ', len(df3[df3.geo_country == item]))

Russia  -  48099
some_unimportant_country  -  856
United States  -  111
Ukraine  -  247
Belarus  -  100


In [77]:
cols_to_encode.append('geo_country')

In [78]:
cols_to_drop.append('visit_time')

In [79]:
for item in df3.geo_city.unique():
    if len(df3[df3.geo_city == item]) / len(df3) >= 0.0018:
        continue
    else:
        df3.loc[df3.geo_city == item, 'geo_city'] = 'some_unimportant_city'
           

In [80]:
for item in df3.geo_city.unique():
    print(item, ' - ', len(df3[df3.geo_city == item]))

Moscow  -  21521
Saint Petersburg  -  7359
Novosibirsk  -  566
Irkutsk  -  340
some_unimportant_city  -  6283
Voronezh  -  419
Yekaterinburg  -  1023
Volgograd  -  100
Domodedovo  -  149
Vladivostok  -  304
Balashikha  -  368
Rostov-on-Don  -  422
Kazan  -  874
Khimki  -  252
Samara  -  723
(not set)  -  1828
Krasnodar  -  925
Zheleznodorozhny  -  150
Krasnoyarsk  -  416
Saratov  -  207
Lipetsk  -  125
Khabarovsk  -  116
Mytishchi  -  165
Nizhny Novgorod  -  595
Pyatigorsk  -  102
Vidnoye  -  168
Tver  -  98
Ufa  -  624
Yaroslavl  -  263
Tula  -  409
Chelyabinsk  -  427
Perm  -  209
Krasnogorsk  -  113
Vladimir  -  90
Sochi  -  295
Grozny  -  346
Stavropol  -  109
Odintsovo  -  172
Izhevsk  -  164
Korolyov  -  112
Podolsk  -  96
Tyumen  -  181
Omsk  -  109
Kaluga  -  96


In [81]:
len(df3.geo_city.unique())

44

In [82]:
cols_to_encode.append('geo_city')

In [83]:
df3 = df3.drop(cols_to_drop, axis=1)

In [84]:
cols_to_encode

['hit_referer',
 'event_category',
 'event_label',
 'utm_source',
 'utm_medium',
 'device_category',
 'device_os',
 'device_brand',
 'resolution',
 'device_browser',
 'geo_country',
 'geo_city']

In [85]:
df3.loc[:, cols_to_encode]

,hit_referer,event_category,event_label,utm_source,utm_medium,device_category,device_os,device_brand,resolution,device_browser,geo_country,geo_city
0,other,sap_search_form,oLRtNHqYKzzRepdgppqs,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,iOS,Apple,mobile_medium,Safari,Russia,Moscow
1,other,card_web,other,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,iOS,Apple,mobile_medium,Safari,Russia,Moscow
2,HbolMJUevblAbkHClEQa,search_form,KEgIdUulHJWFTqlZDBzT,aXQzDWsJuGXeBXexNHjc,referral,mobile,unknown,Xiaomi,mobile_medium,Chrome,Russia,Saint Petersburg
3,HbolMJUevblAbkHClEQa,sub_page_view,KclpemfoHstknWHFiLit,hTjLvqNxGggkGnxSCaTm,tg,mobile,unknown,Apple,mobile_medium,Safari,Russia,Moscow
4,HbolMJUevblAbkHClEQa,card_web,KclpemfoHstknWHFiLit,fDLlAcSmythWSCVMvqvL,(none),mobile,unknown,Apple,mobile_medium,Safari,Russia,Moscow
...,...,...,...,...,...,...,...,...,...,...,...,...
49408,HbolMJUevblAbkHClEQa,sub_button_click,OHfyUfDKdPgBdvelDlfG,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,unknown,Samsung,mobile_medium,Chrome,Russia,Saint Petersburg
49409,HbolMJUevblAbkHClEQa,card_web,KclpemfoHstknWHFiLit,bByPQxmDaMXgpHeypKSM,referral,mobile,unknown,Samsung,mobile_medium,Samsung Internet,Russia,Kazan
49410,other,sub_button_click,other,fDLlAcSmythWSCVMvqvL,(none),mobile,Android,Xiaomi,mobile_medium,Chrome,Russia,Moscow
49411,HbolMJUevblAbkHClEQa,card_web,KclpemfoHstknWHFiLit,NwLFDlNWnYxuLZEAZppl,referral,mobile,unknown,Apple,mobile_medium,Safari,Russia,Grozny


In [86]:
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore', sparse=False)

In [87]:
encoder.fit(df3.loc[:, cols_to_encode])

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(handle_unknown='ignore', sparse=False, sparse_output=False)

In [88]:
encoded_array = encoder.transform(df3.loc[:, cols_to_encode])

In [89]:
feature_names = encoder.get_feature_names_out()
len(feature_names)

2202

In [90]:
encoded_df = pd.DataFrame(encoded_array, columns=feature_names)


,hit_referer_AGjbwsxIYLMEDxvfGWPu,hit_referer_AHveRNjchBposifovhGQ,hit_referer_ASBbDLYyUxSBlxnOZlFi,hit_referer_AjcmNbUKuwGUzvbaajnw,hit_referer_AmAkQrLHCJFZeenuizmO,hit_referer_AmHgnDWCxbQrmwgxhiGd,hit_referer_ApmlIMMchPYvWzNVYuIi,hit_referer_AwPvOUsBoCgbeQZTMTlB,hit_referer_AxUMqnGJHDxlmAAGDgrG,hit_referer_AywhtEOzxbDVFPLcRXlP,...,geo_city_Ufa,geo_city_Vidnoye,geo_city_Vladimir,geo_city_Vladivostok,geo_city_Volgograd,geo_city_Voronezh,geo_city_Yaroslavl,geo_city_Yekaterinburg,geo_city_Zheleznodorozhny,geo_city_some_unimportant_city
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49409,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49411,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [92]:
df3 = df3.drop(cols_to_encode, axis=1)

In [137]:
df3 = df3.join(encoded_df)

In [95]:
df3.columns

Index(['Unnamed: 0', 'hit_number', 'target_action', 'visit_number',
       'day_of_week', 'hit_page_path_greenday', 'hit_page_path_subscription',
       'hit_page_path_general', 'client_number_of_visits', 'camp_succ_rate',
       ...
       'geo_city_Ufa', 'geo_city_Vidnoye', 'geo_city_Vladimir',
       'geo_city_Vladivostok', 'geo_city_Volgograd', 'geo_city_Voronezh',
       'geo_city_Yaroslavl', 'geo_city_Yekaterinburg',
       'geo_city_Zheleznodorozhny', 'geo_city_some_unimportant_city'],
      dtype='object', length=2212)

In [ ]:
df3 = df3.drop('Unnamed: 0.1', axis=1)


In [96]:
df3 = df3.drop('Unnamed: 0', axis=1)

In [97]:
df3.hit_number

0        18
1        31
2         7
3        28
4        31
         ..
49408     4
49409    22
49410    73
49411     3
49412    51
Name: hit_number, Length: 49413, dtype: int64

In [98]:
show_empties_percentage(df3)

% of empties
hit_number  -  0.0
target_action  -  0.0
visit_number  -  0.0
day_of_week  -  0.0
hit_page_path_greenday  -  0.0
hit_page_path_subscription  -  0.0
hit_page_path_general  -  0.0
client_number_of_visits  -  0.0
camp_succ_rate  -  0.0
hit_referer_AGjbwsxIYLMEDxvfGWPu  -  0.0
hit_referer_AHveRNjchBposifovhGQ  -  0.0
hit_referer_ASBbDLYyUxSBlxnOZlFi  -  0.0
hit_referer_AjcmNbUKuwGUzvbaajnw  -  0.0
hit_referer_AmAkQrLHCJFZeenuizmO  -  0.0
hit_referer_AmHgnDWCxbQrmwgxhiGd  -  0.0
hit_referer_ApmlIMMchPYvWzNVYuIi  -  0.0
hit_referer_AwPvOUsBoCgbeQZTMTlB  -  0.0
hit_referer_AxUMqnGJHDxlmAAGDgrG  -  0.0
hit_referer_AywhtEOzxbDVFPLcRXlP  -  0.0
hit_referer_BAMOgnXQWaKcObKyvaVU  -  0.0
hit_referer_BAOSJdLNalSuGQlAHyjn  -  0.0
hit_referer_BDxNaknveJjbqYNnxFJF  -  0.0
hit_referer_BFaSNkTMDShrOstkbyIK  -  0.0
hit_referer_BFhTUIVdGfZJcMZzRlsE  -  0.0
hit_referer_BFmXcdaVpkTlBnOwfcul  -  0.0
hit_referer_BIPScrnstiBsIGzCyYZC  -  0.0
hit_referer_BOTWEpEabDLPchdYxBuD  -  0.0
hit_referer_BPwL

hit_referer_ULkfhKawOIvCrGRiBUIG  -  0.0
hit_referer_UVFYCyyLhgWgfFvpCVdr  -  0.0
hit_referer_UWoKePmNyHypLfhpMKuh  -  0.0
hit_referer_UaVhacPHSEpvJbXwGdtB  -  0.0
hit_referer_UkqtDFGYtENPgjHpawpH  -  0.0
hit_referer_UqdrzaXCSShMhBDtsYqh  -  0.0
hit_referer_UsblUIWwaTOwJttBLsRj  -  0.0
hit_referer_UtikFmIBdSMWuIudRwzl  -  0.0
hit_referer_UxxFvxIxkEKAqNqtnkKV  -  0.0
hit_referer_VCwkVfeTxbsMOOksOkRc  -  0.0
hit_referer_VDZjQhfBCVqPmbpZrOFb  -  0.0
hit_referer_VJEmpKWftTLEgCgFNAuz  -  0.0
hit_referer_VJcjFghQPkMqtpzpYqnZ  -  0.0
hit_referer_VNJHSighoyuhgqUnNunR  -  0.0
hit_referer_VREyNIbgDlzjAxiKuHEq  -  0.0
hit_referer_VTfqxwecLaojEweFlacB  -  0.0
hit_referer_VUdwPJCSpKtxmVWpbKio  -  0.0
hit_referer_VVSsZYpcnWZHZjBNPtTr  -  0.0
hit_referer_VXMxiYvXRgRVztpoiUbj  -  0.0
hit_referer_VYjVGiBBbAAUhwYaKoXT  -  0.0
hit_referer_VcQaSpkEgwLrvWWnUuLz  -  0.0
hit_referer_VgVmpaHvXKWLyZgGPCUk  -  0.0
hit_referer_VgiwSfNuUFMOiyZtiPnA  -  0.0
hit_referer_VizyhCimVozhCaZOrCqF  -  0.0
hit_referer_Vjog

hit_referer_yxdVVyWiWMFaxfdZzmBC  -  0.0
hit_referer_zBKTCFUiXurMYBESygaA  -  0.0
hit_referer_zMyDgvUUqHnnkLIHSlvZ  -  0.0
hit_referer_zUWZklChnvafLhtdxjNM  -  0.0
hit_referer_zYKApxgGYbXsqfqgnIDp  -  0.0
hit_referer_zheKItZOKaFyCuslexDM  -  0.0
hit_referer_zkzCvLqrbyHHsDMMYVKa  -  0.0
hit_referer_zpkCOCjptAAOxrlaKYsq  -  0.0
hit_referer_zpmMwMHCcQCqcuIOYKmO  -  0.0
hit_referer_zsmhkwcIJVJNnTjzyqfx  -  0.0
hit_referer_ztGHbwqZoQQWfTMOmmYm  -  0.0
hit_referer_ztdlQCrIpPehRpaRhhCE  -  0.0
hit_referer_zvOzGHHkQgGNcdtfqMmH  -  0.0
hit_referer_zwMHfIgPspmCDWfJMsiN  -  0.0
hit_referer_zxPaddltVsJZegslUWyX  -  0.0
event_category_active_invitation  -  0.0
event_category_ad_banner  -  0.0
event_category_add_ad  -  0.0
event_category_auth  -  0.0
event_category_card_web  -  0.0
event_category_cars  -  0.0
event_category_chat  -  0.0
event_category_credit_landing_pos  -  0.0
event_category_footer  -  0.0
event_category_greenday_listing_ads  -  0.0
event_category_greenday_sub_button_click  -  0.0


event_label_NSYvWNxepnZEitVokxiF  -  0.0
event_label_NXBDCwxnSBiXptgyFlWq  -  0.0
event_label_NYBsmUuJZKITSwOxosgU  -  0.0
event_label_NaWDEEQoRUFNYvEqftUU  -  0.0
event_label_NbJKiKQmpLxEMXdtHgpm  -  0.0
event_label_NeMmCLSWqbdWqVwDzkyB  -  0.0
event_label_NkHVtgvnntbFyNyUbLFX  -  0.0
event_label_NpbgMkHJkJhacJngCgfd  -  0.0
event_label_NvkcqLWzvzlhjLyHkqhO  -  0.0
event_label_NxqJKRDfnpLugtRTwlCn  -  0.0
event_label_OBWugmWUBwgotoMhlfXF  -  0.0
event_label_OCOqVIjPpbXuNwHSrCCr  -  0.0
event_label_OGHHlafVRngvsfadBYXR  -  0.0
event_label_OHfyUfDKdPgBdvelDlfG  -  0.0
event_label_OHxtAZiXBYGJZPbibrUH  -  0.0
event_label_OHyfVfOZAXLHTvVrwgVk  -  0.0
event_label_OISdoxBdHeJNFotQcSuh  -  0.0
event_label_OLRjhyjlZjrvyszqdNFW  -  0.0
event_label_OLbdmiCYzCkrAwfLrrrP  -  0.0
event_label_OSIAyGajRJpiIFYvCaSy  -  0.0
event_label_OSucmOyBXEpHCUNRuhuN  -  0.0
event_label_OVKizyFisfhLnumivUYu  -  0.0
event_label_OWJVsiWFsRNBmaXySAYC  -  0.0
event_label_OYDJFAlXgbxsQATWDLeT  -  0.0
event_label_OZFY

event_label_dANJnOEVQRkgsZNVaXDb  -  0.0
event_label_dDhNFmEjNFvcMwtGgUWD  -  0.0
event_label_dFgCfMVPXeQbZPMsxaeA  -  0.0
event_label_dHOeSwzgPHgfHaBJNcOu  -  0.0
event_label_dIcESZUEasOwVUQftOJa  -  0.0
event_label_dMTTLsWorzQYucdivRdZ  -  0.0
event_label_dOrYodQcXaZMSHxTbncu  -  0.0
event_label_dQxskWvLpBlEHPDDHcMf  -  0.0
event_label_dTonLzjkSbqqnhoTDyHW  -  0.0
event_label_dWVxnIdwcVAbCBDvhRyW  -  0.0
event_label_dfevlpCPphIktTaTFZHH  -  0.0
event_label_dhHVoZsOeMymgmAkFrrn  -  0.0
event_label_dhQpfStniAwXDivXhpnX  -  0.0
event_label_dlwYatgsQMvLsNApVgcw  -  0.0
event_label_dpSWQSEAzzIbcqrCJnGf  -  0.0
event_label_drCBvwBwWDpBmoLIvecw  -  0.0
event_label_dwYrWfrUnIBYRNpslwBx  -  0.0
event_label_dyyiBvxmOTfbwOFyfoVr  -  0.0
event_label_dzWGgUCKpCYmQxUrvATv  -  0.0
event_label_eCQeLARBMaODZnKATmUS  -  0.0
event_label_eCjLbKfpoxbkADNjlxMS  -  0.0
event_label_eEkLatVAYTkibdzPIyDi  -  0.0
event_label_eIzXqdeOtdYOaAAAIprw  -  0.0
event_label_eJXTNwJSfnRVSlgANEKi  -  0.0
event_label_eMXH

event_label_trdTHjZvBEScVXcyqTPH  -  0.0
event_label_tteMUgcWgwcHXigoWPhP  -  0.0
event_label_tvAsNylZQufwQcsdYLQv  -  0.0
event_label_twWGxtXwyXdbxTezzWuN  -  0.0
event_label_twlfGCnyRPFtCeUKaust  -  0.0
event_label_txQEODFFsHbmWraCujCs  -  0.0
event_label_uALHsEfsoqXQGqpBXIpm  -  0.0
event_label_uCeLTwpcACOiebxkOgrx  -  0.0
event_label_uDrUzDNBnREAvxgZmRmY  -  0.0
event_label_uDrwlhtkwJJbRcbyRaTW  -  0.0
event_label_uEwlpDYJqlSZUvJLFojL  -  0.0
event_label_uIkwUhAhJFPCKlISIQeF  -  0.0
event_label_uIpTcARmYlIzdRfuVgSj  -  0.0
event_label_uNHeUlGVlDcIdpFJssDm  -  0.0
event_label_uQiCDefFLUBJiJJQndtb  -  0.0
event_label_uSQaIOCygmfXizGIreyx  -  0.0
event_label_uTRZnAtrdpqkEZsFuSFG  -  0.0
event_label_uVvQRfksIZbCJNLqpjnD  -  0.0
event_label_uWBOoxMmhlyrDZWDtQVU  -  0.0
event_label_uWlpcRmOTEwAHOgZxbbg  -  0.0
event_label_uYvxxYHWgnLOTFxfDHQI  -  0.0
event_label_uZZupzejuxHtxpsxrlUG  -  0.0
event_label_ubetsuajVITuAlZAlwIT  -  0.0
event_label_ucvgMtqkkVkpRcaYnvuF  -  0.0
event_label_udVu

device_browser_Safari  -  0.0
device_browser_Safari (in-app)  -  0.0
device_browser_Samsung Internet  -  0.0
device_browser_UC Browser  -  0.0
device_browser_YaBrowser  -  0.0
device_browser_[FBAN  -  0.0
geo_country_Belarus  -  0.0
geo_country_Russia  -  0.0
geo_country_Ukraine  -  0.0
geo_country_United States  -  0.0
geo_country_some_unimportant_country  -  0.0
geo_city_(not set)  -  0.0
geo_city_Balashikha  -  0.0
geo_city_Chelyabinsk  -  0.0
geo_city_Domodedovo  -  0.0
geo_city_Grozny  -  0.0
geo_city_Irkutsk  -  0.0
geo_city_Izhevsk  -  0.0
geo_city_Kaluga  -  0.0
geo_city_Kazan  -  0.0
geo_city_Khabarovsk  -  0.0
geo_city_Khimki  -  0.0
geo_city_Korolyov  -  0.0
geo_city_Krasnodar  -  0.0
geo_city_Krasnogorsk  -  0.0
geo_city_Krasnoyarsk  -  0.0
geo_city_Lipetsk  -  0.0
geo_city_Moscow  -  0.0
geo_city_Mytishchi  -  0.0
geo_city_Nizhny Novgorod  -  0.0
geo_city_Novosibirsk  -  0.0
geo_city_Odintsovo  -  0.0
geo_city_Omsk  -  0.0
geo_city_Perm  -  0.0
geo_city_Podolsk  -  0.0
geo

0

In [99]:
for feature in df3.columns:
    if df3[feature].dtype != 'O':
        print(feature, ' - ', df3[feature].dtype)

hit_number  -  int64
target_action  -  int64
visit_number  -  int64
day_of_week  -  int64
hit_page_path_greenday  -  int64
hit_page_path_subscription  -  int64
hit_page_path_general  -  int64
client_number_of_visits  -  int64
camp_succ_rate  -  float64
hit_referer_AGjbwsxIYLMEDxvfGWPu  -  float64
hit_referer_AHveRNjchBposifovhGQ  -  float64
hit_referer_ASBbDLYyUxSBlxnOZlFi  -  float64
hit_referer_AjcmNbUKuwGUzvbaajnw  -  float64
hit_referer_AmAkQrLHCJFZeenuizmO  -  float64
hit_referer_AmHgnDWCxbQrmwgxhiGd  -  float64
hit_referer_ApmlIMMchPYvWzNVYuIi  -  float64
hit_referer_AwPvOUsBoCgbeQZTMTlB  -  float64
hit_referer_AxUMqnGJHDxlmAAGDgrG  -  float64
hit_referer_AywhtEOzxbDVFPLcRXlP  -  float64
hit_referer_BAMOgnXQWaKcObKyvaVU  -  float64
hit_referer_BAOSJdLNalSuGQlAHyjn  -  float64
hit_referer_BDxNaknveJjbqYNnxFJF  -  float64
hit_referer_BFaSNkTMDShrOstkbyIK  -  float64
hit_referer_BFhTUIVdGfZJcMZzRlsE  -  float64
hit_referer_BFmXcdaVpkTlBnOwfcul  -  float64
hit_referer_BIPScrnstiBsIGz

In [100]:
cols_to_scale = []

In [101]:
cols_to_scale.append('camp_succ_rate')
cols_to_scale.append('client_number_of_visits')
cols_to_scale.append('visit_number')
cols_to_scale.append('hit_number')

In [106]:
scaler = StandardScaler()

In [107]:
scaled_features = scaler.fit_transform(df3.loc[:,cols_to_scale])

In [108]:
scaled_feature_names = scaler.get_feature_names_out()

In [109]:
scaled_feature_names

array(['camp_succ_rate', 'client_number_of_visits', 'visit_number',
       'hit_number'], dtype=object)

In [110]:
scaled_df = pd.DataFrame(scaled_features, columns=scaled_feature_names)

In [111]:
df3 = df3.drop(cols_to_scale, axis=1)

In [112]:
df3 = df3.join(scaled_df)

In [113]:
df3

,target_action,day_of_week,hit_page_path_greenday,hit_page_path_subscription,hit_page_path_general,hit_referer_AGjbwsxIYLMEDxvfGWPu,hit_referer_AHveRNjchBposifovhGQ,hit_referer_ASBbDLYyUxSBlxnOZlFi,hit_referer_AjcmNbUKuwGUzvbaajnw,hit_referer_AmAkQrLHCJFZeenuizmO,...,geo_city_Volgograd,geo_city_Voronezh,geo_city_Yaroslavl,geo_city_Yekaterinburg,geo_city_Zheleznodorozhny,geo_city_some_unimportant_city,camp_succ_rate,client_number_of_visits,visit_number,hit_number
0,0,5,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.077271,1.059871,-0.131017,-0.188771
1,0,3,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.077271,1.059871,-0.131017,0.270748
2,0,5,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122651,-0.279175,-0.053741,-0.577595
3,0,0,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.316820,-0.279175,-0.131017,0.164705
4,0,3,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122651,-0.279175,-0.131017,0.270748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49408,0,5,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.456333,-0.279175,-0.131017,-0.683638
49409,0,4,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122651,-0.279175,0.023535,-0.047381
49410,0,1,0,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122651,-0.279175,-0.131017,1.755349
49411,0,3,0,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.122651,-0.279175,0.023535,-0.718986


# prediction time, bitch!

In [115]:
x = df3.drop('target_action', axis=1)
y = df3['target_action']

In [116]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [117]:
rf = RandomForestClassifier()

In [118]:
rf.fit(x_train, y_train)

RandomForestClassifier()

In [119]:
predicted_train = rf.predict(x_train)

In [120]:
predicted_test = rf.predict(x_test)

In [121]:
accuracy_score(y_train, predicted_train)

1.0

In [122]:
accuracy_score(y_test, predicted_test)

0.9957501349163519

In [123]:
tst = rf.predict_proba(x_train)
tst

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [128]:
y_pred_probs = rf.predict_proba(x_train)[:,1]

In [130]:
roc_auc_score(y_train, rf.predict_proba(x_train)[:,1])

1.0

In [131]:
roc_auc_score(y_test, rf.predict_proba(x_test)[:,1])

0.9977515654414948